# HyLang

Hy is a "dialect of Lisp that's embedded in Python"

- [Hy Learn X in Y minutes](https://learnxinyminutes.com/docs/hy/)
- [Stable docs](http://docs.hylang.org/en/stable/index.html)
- [Github](https://github.com/hylang/hy)
- [hy magic for ipython](https://github.com/yardsale8/hymagic)

In [2]:
import hy
hy.__version__
# NB. This is the pypi version, there is some talk on the #hy irc channel about a 
# new release, but it's certainly usable how it is at the moment

'0.11.1'

In [3]:
%load_ext hymagic

## 1. Python Integration

Hy uses custom data structures (that pprint doesn't understand), which means it's necessary to flatten everything back to fundamental list/tuple types if you want to show what's in a complex quoted structure using python. 

Not sure if this is really required (or even sensible), but it can be helpful for debugging.

In [117]:
def hy2py(expr):
    if type(expr) == hy.models.expression.HyList:
        return [ hy2py(se) for se in expr ]
    elif type(expr) == hy.models.expression.HyExpression:
        return tuple(hy2py(se) for se in expr)
    elif type(expr) == list:
        return [ hy2py(se) for se in expr]
    else:
        return expr
    
def showhy(expr):
    print expr

## 2. Example use case

[pydubins](https://github.com/AndrewWalker/pydubins) uses cython to wrap c code thats numeric in nature - nothing particularly complex, it wraps up the solutions to six equations, but there are enough corner cases to make bugs likely.  One nice feature would be to generate (nice) c, python and matlab versions of the code. The challenge was also that if I found a bug, I'd need to rewrite the code in both Python and C to fix the issue.  

So the sample use-case for hy is to show you can generate both python and c code from hy (polyglot ftw) 

### 2.1 Porting a function from Python to Hy

Total time about 3 hours 

- I'd read the little schemer, and had a few lisp books lying around that I'd been getting ideas from for a while, but this is the first lisp code I've ever written of this size
- interestingly most of the this was because of looking at the incorrect version of the hy docs (for latest rather than stable). The structure of let will change in the next release of hy

In [5]:
%%hylang
(import [math [cos sin atan2 acos sqrt pi]])

(defun mod2pi [x]
    (% x (* 2 pi)))

(defun minus [x] (* -1 x))

(defun dubins_LSL [alpha beta d]
    (let [[ca (cos alpha)]
          [cb (cos beta)]
          [sa (sin alpha)]
          [sb (sin beta)]
          [c_ab (cos (- alpha beta))]
          [tmp0 (+ d sa (minus sb))]
          [psq (+ 2 (* d d) (* -2 c_ab) (* 2 d (- sa sb)))]]
        (if (>= psq 0)
            (let [[tmp1 (atan2 (- ca cb) tmp0)]
                  [t (mod2pi (- tmp1 alpha))]
                  [p (sqrt psq)]
                  [q (mod2pi (- beta tmp1))]]
                 [t p q])
            nil)))

(print (dubins_LSL 0 0.1 2))

[0.0026291501253711603, 1.9001731507566977, 0.09737084987462885]


### 2.2 Generalizing that function

So the next step is to try to get access to the s-expressions that make up the function - you can do that by returning a quoted function.

- This trick is pretty obvious, and was the main reason I'd been considering hy for doing this
- What was less obvious is how to eval to go from that quoted list back to a function. Got lots of errors about "unquoting at the top level" which weren't helpful (for a novice) - in retrospect there are a few examples in the hy documentation that show how to do this.

In [8]:
%%hylang

(defun mkfn []
    `(lambda [alpha beta d]
        (let [[ca (cos alpha)]
              [cb (cos beta)]
              [sa (sin alpha)]
              [sb (sin beta)]
              [c_ab (cos (- alpha beta))]
              [tmp0 (+ d sa (minus sb))]
              [psq (+ 2 (* d d) (* -2 c_ab) (* 2 d (- sa sb)))]]
            (if (>= psq 0)
                (let [[tmp1 (atan2 (- ca cb) tmp0)]
                      [t (mod2pi (- tmp1 alpha))]
                      [p (sqrt psq)]
                      [q (mod2pi (- beta tmp1))]]
                     [t p q])
                nil))))

(setv myfn (mkfn))

### 2.2a Sanity Check

Hy includes some goodies like converting to python. Let's try that and show it works

In [10]:
%%hylang
(print (disassemble myfn True))

def _hy_anon_fn_3(alpha, beta, d):

    def _hy_anon_fn_2():
        ca = cos(alpha)
        cb = cos(beta)
        sa = sin(alpha)
        sb = sin(beta)
        c_ab = cos((alpha - beta))
        tmp0 = ((d + sa) + minus(sb))
        psq = (((2L + (d * d)) + ((-2L) * c_ab)) + ((2L * d) * (sa - sb)))
        if (psq >= 0L):

            def _hy_anon_fn_1():
                tmp1 = atan2((ca - cb), tmp0)
                t = mod2pi((tmp1 - alpha))
                p = sqrt(psq)
                q = mod2pi((beta - tmp1))
                return [t, p, q]
            _hy_anon_var_1 = _hy_anon_fn_1()
        else:
            _hy_anon_var_1 = None
        return _hy_anon_var_1
    return _hy_anon_fn_2()


### 2.3 Code Injection

In the current version of the C code, a c-preprocessor macro injects a heap of common code into each function, just to make things managable. Because we're writing in a lisp, we can do heaps better than that - we can just mutate the code.

In [88]:
%%hylang

(defun mkfn [prelude]
    `(lambda [alpha beta d]
        (let [~@prelude             
              [tmp0 (+ d sa (minus sb))]
              [psq (+ 2 (* d d) (* -2 c_ab) (* 2 d (- sa sb)))]]
            (if (>= psq 0)
                (let [[tmp1 (atan2 (- ca cb) tmp0)]
                      [t (mod2pi (- tmp1 alpha))]
                      [p (sqrt psq)]
                      [q (mod2pi (- beta tmp1))]]
                     (list t p q))
                nil))))

(setv lsl (mkfn `[[ca (cos alpha)]
        [cb (cos beta)]
        [sa (sin alpha)]
        [sb (sin beta)]
        [c_ab (cos (- alpha beta))]]))

(hy2py lsl)

(u'lambda',
 [u'alpha', u'beta', u'd'],
 (u'let',
  [[u'ca', (u'cos', u'alpha')],
   [u'cb', (u'cos', u'beta')],
   [u'sa', (u'sin', u'alpha')],
   [u'sb', (u'sin', u'beta')],
   [u'c_ab', (u'cos', (u'-', u'alpha', u'beta'))],
   [u'tmp0', (u'+', u'd', u'sa', (u'minus', u'sb'))],
   [u'psq',
    (u'+',
     2L,
     (u'*', u'd', u'd'),
     (u'*', -2L, u'c_ab'),
     (u'*', 2L, u'd', (u'-', u'sa', u'sb')))]],
  (u'if',
   (u'>=', u'psq', 0L),
   (u'let',
    [[u'tmp1', (u'atan2', (u'-', u'ca', u'cb'), u'tmp0')],
     [u't', (u'mod2pi', (u'-', u'tmp1', u'alpha'))],
     [u'p', (u'sqrt', u'psq')],
     [u'q', (u'mod2pi', (u'-', u'beta', u'tmp1'))]],
    (u'list', u't', u'p', u'q')),
   u'None')))

### 2.4 Code Extraction

Add a top level form to the lambda that describes all the declarations present in the function. This isn't necessary for lisp - but is a necessary precursor for generating C90 style code.

In [142]:
%%hylang

(defun declaration [expr]
    (cond [(numeric? expr) ()]
          [(empty? expr) ()]
          [(nil? expr) ()]
          [(= `lambda (car expr)) 
               (declaration (nth expr 2))]
          [(= `let (car expr)) 
               (+
                    (nth expr 1)
                    (declaration (nth expr 2)))]
          [(= `if  (car expr)) 
               (declaration (nth expr 2))]
          [True ()]))

(defun injectdecls [expr]
    (let [[vdecls (list (map first (declaration expr)))]]
        `(lambda ~(nth expr 1)
             (vardecls ~vdecls ~(nth expr 2)))))

(defun rewriteletterm [term]
    `(setv ~(car term) ~(car (cdr term))))

(defun rewritelet [expr]
    (if (coll? expr)
        (if (= `let (car expr))
            `(do 
                ~(list (map rewriteletterm (nth expr 1))) 
                ~(rewritelet (nth expr 2)))
            (list (map rewritelet expr)))
        expr))

(hy2py (rewritelet (injectdecls lsl)))

[u'lambda',
 [u'alpha', u'beta', u'd'],
 [u'vardecls',
  [u'ca',
   u'cb',
   u'sa',
   u'sb',
   u'c_ab',
   u'tmp0',
   u'psq',
   u'tmp1',
   u't',
   u'p',
   u'q'],
  (u'do',
   [(u'setv', u'ca', (u'cos', u'alpha')),
    (u'setv', u'cb', (u'cos', u'beta')),
    (u'setv', u'sa', (u'sin', u'alpha')),
    (u'setv', u'sb', (u'sin', u'beta')),
    (u'setv', u'c_ab', (u'cos', (u'-', u'alpha', u'beta'))),
    (u'setv', u'tmp0', (u'+', u'd', u'sa', (u'minus', u'sb'))),
    (u'setv',
     u'psq',
     (u'+',
      2L,
      (u'*', u'd', u'd'),
      (u'*', -2L, u'c_ab'),
      (u'*', 2L, u'd', (u'-', u'sa', u'sb'))))],
   [u'if',
    [u'>=', u'psq', 0L],
    (u'do',
     [(u'setv', u'tmp1', (u'atan2', (u'-', u'ca', u'cb'), u'tmp0')),
      (u'setv', u't', (u'mod2pi', (u'-', u'tmp1', u'alpha'))),
      (u'setv', u'p', (u'sqrt', u'psq')),
      (u'setv', u'q', (u'mod2pi', (u'-', u'beta', u'tmp1')))],
     [u'list', u't', u'p', u'q']),
    u'None'])]]

In [174]:
%%hylang
;symbol?
(defun rewriterhs [expr]
    (cond [(not (coll? expr)) 
           (str expr)]
          [(= `* (car expr)) 
           (.format "({0})" (.join " * " (map rewriterhs (cdr expr))))]
          [(= `+ (car expr)) 
           (.format "({0})" (.join " + " (map rewriterhs (cdr expr))))]
          [(= `- (car expr)) 
           (.format "({0} - {1})" (nth expr 1) (nth expr 2))]
          [True expr]))
    

(rewriterhs `((+ 2 (* d d) (* -2 c_ab) (* 2 d (- sa sb)))))

((u'+' 2L (u'*' u'd' u'd') (u'*' -2L u'c_ab') (u'*' 2L u'd' (u'-' u'sa' u'sb'))))

In [164]:
"({0})".format('oo')

'(oo)'

### 2.2b Restatement of Goal

Rather than the code in the previous section we really want:

In [ ]:
; rewrite rule 
;   (let [[u v] [w x]] q) -> 
;      (do (set u v) (set w x) q nil)

;   (setv x y)
;   x = y;

; NEED TO CHECK PRECENDENCE
;   (minus x)
;   -x""

;  (- a b)
;  (a - b)

; (+ x0 x1 x2 ... xn)
;  (x0 + x1 + x2 ... + xn)

; PHASE 1
    
(defun mkfn []
    `(lambda [alpha beta d]
       (decls [ca cb sa sb c_ab tmp0 psq tmp1 t p q]
           (do
              (setv c_ab (cos (- alpha beta)))
              (setv tmp0 (+ d sa (minus sb)))
              (setv psq (+ 2 (* d d) (* -2 c_ab) (* 2 d (- sa sb))))
              (if (>= psq 0)
                  (do
                      (setv tmp1 (atan2 (- ca cb) tmp0))
                      (setv t (mod2pi (- tmp1 alpha)))
                      (setv p (sqrt psq))
                      (setv q (mod2pi (- beta tmp1)))
                      nil)
                  nil)
            nil))))


; PHASE 2

; (if q c t nil) ->
;
; (do
;   (println (concat "if(" (cexpr c) ")"))
;   (println "{")
;   (cexpr t)
;   (println "}"))

    
(defun mkfn []
    `(lambda [alpha beta d]
       (decls [ca cb sa sb c_ab tmp0 psq tmp1 t p q]
           (do
              (println "c_ab = cos(alpha - beta)")
              (println "tmp0 = d + sa + -sb")
              (println "psq = 2 + (d * d) + (-2 * c_ab) + (2 * d * (sa - sb))")
              (if (>= psq 0)
                  (do
                      (setv tmp1 (atan2 (- ca cb) tmp0))
                      (setv t (mod2pi (- tmp1 alpha)))
                      (setv p (sqrt psq))
                      (setv q (mod2pi (- beta tmp1)))
                      nil)
                  nil)
            nil))))


### 2.3 Matching Expression fragments (Python)

One of the things lisp is great at is matching part of expressions while you're trying to walk a tree. This (ugly) python 2 code shows what this might look like (nb, yield-from would make this slightly less bad in python 3)

In [40]:
def match_let(expr):
    if expr[0] == u'lambda':
        for m in match_let(expr[2]):
            yield m
    elif expr[0] == u'if':
        for m in match_let(expr[2]):
            yield m
        for m in match_let(expr[3]):
            yield m
    elif expr[0] == u'let':
        yield expr[1]
        for m in match_let(expr[2]):
            yield m

def declarations(expr):
    for e in match_let(myfn):
        for item in e:
            yield item[0]

from  pprint import pprint
print 'double ' + ', '.join(declarations(myfn)) + ';'

double ca, cb, sa, sb, c_ab, tmp0, psq, tmp1, t, p, q;


In [14]:
%%hylang

(nth (nth myfn 2) 1)


[[u'ca' (u'cos' u'alpha')] [u'cb' (u'cos' u'beta')] [u'sa' (u'sin' u'alpha')] [u'sb' (u'sin' u'beta')] [u'c_ab' (u'cos' (u'-' u'alpha' u'beta'))] [u'tmp0' (u'+' u'd' u'sa' (u'minus' u'sb'))] [u'psq' (u'+' 2L (u'*' u'd' u'd') (u'*' -2L u'c_ab') (u'*' 2L u'd' (u'-' u'sa' u'sb')))]]

In [66]:
%%hylang

(defun rewrite [e] 
    (cond 
        [(= `lambda (car e)) `(lambda ~(nth e 1) ~(rewrite (nth e 2)))]
        [(= `let (car e) `(do
                              (setv u ))]
        [True e]))

(rewrite `(lambda [x] 
              (let [[u (cos x)]
                    [v (sin x)]]
                  (+ u v))))

(u'lambda' [u'x'] (u'let' [[u'u' (u'cos' u'x')] [u'v' (u'sin' u'x')]] (u'+' u'u' u'v')))

In [77]:
%%hylang
(last `(1 2 3))
;(reduce cons `(1 2 3) nil)

3L

In [100]:
%%hylang
(defun foldr [fun seq end]
    (if (nil? seq)
            end
            (fun (car seq) (foldr fun end (cdr seq)))))

(foldr cons `[1 2 3] nil)

(eval `(1 2 3))

TypeError: 'long' object is not callable